In [48]:
import os
import pandas as pd
from fastbook import *
from fastai.tabular.all import *
import sklearn


traindf = pd.read_csv('C:/Users/jsult/Desktop/spacetitanic3/train.csv',low_memory=False)
testdf = pd.read_csv('C:/Users/jsult/Desktop/spacetitanic3/test.csv',low_memory= False)
togeth = pd.concat([testdf,traindf])

Need to set to categories after !!


In [50]:
def split_group_number(df):
    df['Group'] = df['PassengerId'].apply(lambda x: x.split("_")[0] if pd.notna(x) else pd.NA)
    df.Group = df.Group.astype('float')
    df['GroupNumber'] = df['PassengerId'].apply(lambda x: x.split("_")[1] if pd.notna(x) else pd.NA)
    df.GroupNumber = df.GroupNumber.astype('float')
    return df

def split_cabin(df):
    df['CabinDeck'] = df['Cabin'].apply(lambda x: x.split("/" )[0] if pd.notna(x) else pd.NA)
    custom_order = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'T']
    df['CabinDeck'] = pd.Categorical(df['CabinDeck'], categories=custom_order, ordered=True)
    df['CabinSide'] = df['Cabin'].apply(lambda x: x.split("/")[2] if pd.notna(x) else pd.NA)
    df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split("/")[1]) if pd.notna(x) else pd.NA)
    return df

def split_names(df):
    # Create new columns for first name and last name
    df['FirstName'] = df['Name'].apply(lambda x: x.split()[0] if pd.notna(x) else pd.NA)
    df['LastName'] = df['Name'].apply(lambda x: x.split()[1] if pd.notna(x) else pd.NA)
    return df

def fill_home_planet_by_last_name(df):
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            matching_rows = df[df['LastName'] == row['LastName']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            if not non_nan_planet.empty:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = planet_to_fill
    return df

def fill_home_planet_by_group(df):
    for index, row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            matching_rows = df[df['Group'] == row['Group']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            if not non_nan_planet.empty:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = planet_to_fill
    return df

def home_planet_from_cabindeck_abc(df):
    df.loc[df['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'] = df.loc[df['CabinDeck'].isin(['A', 'B', 'C']), 'HomePlanet'].fillna('Europa')
    return df

def home_planet_from_cabindeck_g(df):
    df.loc[df['CabinDeck']== 'G', 'HomePlanet'] = df.loc[df['CabinDeck'] == 'G', 'HomePlanet'].fillna('Earth')
    return df

def cabin_dm(df):
    df = split_group_number(df)
    df = split_cabin(df)
    df = split_names(df)
    df = df.sort_values(by = ['Group','GroupNumber'])
    df.reset_index(inplace = True)
    df.drop(columns = 'index',inplace = True)
    df = fill_home_planet_by_group(df)
    df = fill_home_planet_by_last_name(df)
    df = home_planet_from_cabindeck_abc(df)
    df = home_planet_from_cabindeck_g(df)
    return df


In [51]:
togeth = cabin_dm(togeth)

In [52]:
def indexfromid(id):
    return togeth[togeth.PassengerId == id].index.item()

def idfromindex(index):
    return togeth.loc[index].PassengerId
    
def cab_fill(cabfilllist):
    for item in cabfilllist:
        ind = indexfromid(item[0])
        togeth.at[ind,'Cabin'] = item[1]
    

In [53]:
temp = togeth.copy()

In [54]:
togeth = temp.copy()

In [55]:
togeth.at[indexfromid('3034_01'),'Cabin'] = 'B/98/P'
togeth.at[indexfromid('3053_01'),'Cabin'] = 'B/99/P'
togeth.at[indexfromid('4637_01'),'Cabin'] = 'E/300/S'
togeth.at[indexfromid('4652_01'),'Cabin'] = 'E/301/S'
togeth.at[indexfromid('4953_01'),'Cabin'] = 'C/193/S'
togeth.at[indexfromid('4974_02'),'Cabin'] = 'A/44/P'
togeth.at[indexfromid('5480_01'),'Cabin'] = 'C/214/S'
togeth.at[indexfromid('5642_02'),'Cabin'] = 'B/216/S'
togeth.at[indexfromid('9223_01'),'Cabin'] = 'F/1785/S'
togeth.at[indexfromid('9223_02'),'Cabin'] = 'F/1785/S'


In [67]:
decks_by_homeplanet = {'Europa':list(togeth[togeth.HomePlanet == 'Europa'].CabinDeck.dropna().unique()),
    'Earth':list(togeth[togeth.HomePlanet == 'Earth'].CabinDeck.dropna().unique()),
    'Mars':list(togeth[togeth.HomePlanet == 'Mars'].CabinDeck.dropna().unique())}

def caboptionsbyplanet(planet,df):
    df = split_cabin(df)
    decks = decks_by_homeplanet[planet]
    sides = {'P':0,'S':0}
    caboptions = {}
    for i,row in df[df.HomePlanet == planet].iterrows():
        if pd.isna(row.Cabin):
            bef = df.iloc[:i].dropna(subset=['Cabin'])
            aft = df.iloc[i+1:].dropna(subset=['Cabin'])
            for deck in decks:
                for side in sides.keys():
                    before = bef[(bef.CabinDeck == deck) & (bef.CabinSide == side)]
                    after = aft[(aft.CabinDeck == deck) & (aft.CabinSide == side)]
                    if len(before) == 0:
                        minn = -2
                    else:
                        minn = max(list(before.CabinNum))
                    if len(after) == 0:
                        maxx = -2
                    else:
                        maxx = min(list(after.CabinNum))
                    if row.PassengerId in caboptions:
                        caboptions[row.PassengerId].append([minn,maxx,deck,side])
                    else:
                        caboptions[row.PassengerId] = [[minn,maxx,deck,side]]
    return caboptions, df

def one_option_solo_group(potential_spots,df):
    cabin_to_fill = []
    for passenger in potential_spots:
        if len(df[df.Group == int(passenger[:4])]) == 1:
            ind = -1
            for i,decks in enumerate(potential_spots[passenger]):
                if decks[1] - decks[0] != 1:
                    if ind == -1:
                        ind = i
                    elif decks[1] != 0:
                        ind = -2
            if ind != -2 and ind != -1:
                #check this line
                cabin_to_fill.append([passenger, potential_spots[passenger][ind][2] + "/" + str(potential_spots[passenger][ind][0] + 1) +"/" + potential_spots[passenger][ind][3]])
    return cabin_to_fill

def no_options(potential_spots,df):
        #no options
    cabin_to_fill = []
    for passenger in potential_spots:
        count = 0
        no_potential_spots = True
        for decks in potential_spots[passenger]:
            if (decks[1] - decks[0] > 1 and decks[1] != 0):
                no_potential_spots = False
        if no_potential_spots:
            if len(list(df[df.Group == int(passenger[:4])].dropna(subset = 'Cabin').Cabin.unique())) == 1:
                cabin_to_fill.append([passenger,list(df[df.Group == int(passenger[:4])].dropna(subset = 'Cabin').Cabin.unique())[0]])
    return cabin_to_fill
 
def only_one_thatll_fit(potential_spots,df):
    cabin_to_fill = []
    for i1,passenger in enumerate(potential_spots.keys()):            
        for i2, cabins in enumerate(potential_spots[passenger]):
            if cabins[1] != 0 and cabins[1] == cabins[0] + 2:
                good = True
                low = i1 - 1
                high = i1  + 1
                lowchecking = True
                highchecking = True
                while lowchecking and low >= 0:
                    option = potential_spots[list(potential_spots.keys())[low]]
                    for choices in option:
                        if choices[2] == cabins[2] and choices[3] == cabins[3]:
                            lowchecking = False
                            if choices[1] >= cabins[0] + 1:
                                good = False
                    low -=1
                while highchecking and good and high <= len(potential_spots.keys()) -1 :
                    option = potential_spots[list(potential_spots.keys())[high]]
                    for choices in option:
                        if choices[2] == cabins[2] and choices[3] == cabins[3]:
                            highchecking = False
                            if choices[0] <= cabins[1] - 1:
                                good = False
                    high += 1                                   
                if good:
                    cabin_to_fill.append([passenger, cabins[2] + "/" + str(cabins[0] + 1) +"/" + cabins[3]])
    return cabin_to_fill

def cab_filler(df):
    
    for planet in ['Europa','Mars','Earth']:
        planet_cab_options, df = caboptionsbyplanet(planet,df)
        cab_fill(no_options(planet_cab_options,df))
    
    
    for planet in ['Europa','Mars','Earth']:
        planet_cab_options, df = caboptionsbyplanet(planet,df)
        cab_fill(one_option_solo_group(planet_cab_options,df))


    europacaboptions = caboptionsbyplanet('Europa',df)[0]
    marscaboptions = caboptionsbyplanet('Mars',df)[0]
    earthcaboptions = caboptionsbyplanet('Earth',df)[0]

    indexlist = [indexfromid(x) for x in list(europacaboptions.keys())] + [indexfromid(x) for x in list(marscaboptions.keys())] + [indexfromid(x) for x in list(earthcaboptions.keys())]
    indexlist.sort()
    idlist = [idfromindex(x) for x in indexlist]
    global_cab_options = {}
    for id in idlist:
        if id in europacaboptions:
            global_cab_options[id] = europacaboptions[id]
        elif id in marscaboptions:
            global_cab_options[id] = marscaboptions[id]
        else:
            global_cab_options[id] = earthcaboptions[id]

    cab_fill(only_one_thatll_fit(global_cab_options,df))

    
    return global_cab_options


  
    


In [68]:
glob = cab_filler(togeth)

In [69]:
togeth.isna().sum()

PassengerId        0
HomePlanet        10
CryoSleep        310
Cabin             55
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name             294
Transported     4277
Group              0
GroupNumber        0
CabinDeck         55
CabinSide         55
CabinNum          55
FirstName        294
LastName         294
dtype: int64

In [59]:
europacaboptions = caboptionsbyplanet('Europa',togeth)[0]
marscaboptions = caboptionsbyplanet('Mars',togeth)[0]
earthcaboptions = caboptionsbyplanet('Earth',togeth)[0]

indexlist = [indexfromid(x) for x in list(europacaboptions.keys())] + [indexfromid(x) for x in list(marscaboptions.keys())] + [indexfromid(x) for x in list(earthcaboptions.keys())]
indexlist.sort()
idlist = [idfromindex(x) for x in indexlist]
global_cab_options = {}
for id in idlist:
    if id in europacaboptions:
        global_cab_options[id] = europacaboptions[id]
    elif id in marscaboptions:
        global_cab_options[id] = marscaboptions[id]
    else:
        global_cab_options[id] = earthcaboptions[id]

for item in global_cab_options:
    print(item,global_cab_options[item])

0244_01 [[52, 53, 'F', 'P'], [46, 47, 'F', 'S'], [7, 8, 'D', 'P'], [7, 8, 'D', 'S'], [11, 12, 'E', 'P'], [15, 16, 'E', 'S']]
0278_01 [[61, 62, 'F', 'P'], [52, 53, 'F', 'S'], [44, 45, 'G', 'P'], [39, 40, 'G', 'S'], [14, 15, 'E', 'P'], [17, 18, 'E', 'S']]
0293_01 [[12, 14, 'B', 'P'], [7, 8, 'B', 'S'], [2, 3, 'A', 'P'], [3, 4, 'A', 'S'], [6, 7, 'C', 'P'], [12, 14, 'C', 'S'], [10, 11, 'D', 'P'], [10, 11, 'D', 'S'], [16, 17, 'E', 'P'], [17, 18, 'E', 'S'], [-2, 0, 'T', 'P'], [-2, 0, 'T', 'S']]
0310_01 [[12, 14, 'B', 'P'], [8, 9, 'B', 'S'], [3, 4, 'A', 'P'], [3, 4, 'A', 'S'], [7, 8, 'C', 'P'], [12, 14, 'C', 'S'], [11, 12, 'D', 'P'], [10, 11, 'D', 'S'], [16, 17, 'E', 'P'], [19, 20, 'E', 'S'], [-2, 0, 'T', 'P'], [-2, 0, 'T', 'S']]
0348_02 [[76, 77, 'F', 'P'], [64, 65, 'F', 'S'], [12, 13, 'D', 'P'], [10, 11, 'D', 'S'], [19, 22, 'E', 'P'], [21, 22, 'E', 'S']]
0364_02 [[81, 82, 'F', 'P'], [68, 69, 'F', 'S'], [12, 13, 'D', 'P'], [10, 11, 'D', 'S'], [19, 22, 'E', 'P'], [22, 23, 'E', 'S']]
0374_02 [[

In [43]:
togeth[togeth.Group == 5480]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
7638,5480_01,Europa,False,NaN,TRAPPIST-1e,37.0,False,31.0,9307.0,0.0,904.0,11213.0,Avijava Ellcefulve,False,5480.0,1.0,NaN,<NA>,<NA>,Avijava,Ellcefulve


In [41]:
togeth[togeth.Cabin == 'G/832/S']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
7127,5098_03,Earth,True,G/832/S,55 Cancri e,2.0,False,0.0,0.0,0.0,0.0,0.0,Delody Kellyons,NaN,5098.0,3.0,G,S,832,Delody,Kellyons


In [39]:
togeth[togeth.PassengerId == '4974_02']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
6952,4974_02,Europa,True,NaN,55 Cancri e,45.0,False,0.0,0.0,0.0,0.0,NaN,Lesat Vendeck,True,4974.0,2.0,NaN,<NA>,<NA>,Lesat,Vendeck


In [38]:
(togeth[(togeth.CabinDeck == 'B') & (togeth.CabinSide == 'P')].FoodCourt > 0).sum()

215

In [39]:
(togeth[(togeth.CabinDeck == 'C') & (togeth.CabinSide == 'S')].FoodCourt > 0).sum()

306

In [41]:
togeth[togeth.Group == 293]

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
404,0293_01,Europa,True,NaN,TRAPPIST-1e,47.0,False,0.0,0.0,0.0,0.0,0.0,Tauxon Suptibler,NaN,293.0,1.0,NaN,<NA>,<NA>,Tauxon,Suptibler


In [26]:
togeth[togeth.PassengerId == '0310_01']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
421,0310_01,Europa,False,NaN,TRAPPIST-1e,67.0,False,NaN,230.0,0.0,4476.0,241.0,Naviton Coudered,False,310.0,1.0,NaN,<NA>,<NA>,Naviton,Coudered


In [25]:
togeth[togeth.PassengerId == '0293_01']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Group,GroupNumber,CabinDeck,CabinSide,CabinNum,FirstName,LastName
404,0293_01,Europa,True,NaN,TRAPPIST-1e,47.0,False,0.0,0.0,0.0,0.0,0.0,Tauxon Suptibler,NaN,293.0,1.0,NaN,<NA>,<NA>,Tauxon,Suptibler


In [70]:
def print_rows_with_different_group(df):
    unique_cabins = df['Cabin'].unique()
    
    for cabin in unique_cabins:
        group_list = df[df['Cabin'] == cabin]['Group'].unique()
        
        if len(group_list) > 1:
            print(cabin)

# Call the function with your DataFrame
print_rows_with_different_group(togeth)


In [71]:
newdict = {}
for item in togeth.iterrows():
    item = item[1]
    if not pd.isna(item.Cabin):
        newdict[item.PassengerId] = item.Cabin
        

In [72]:
import pickle
with open('idtocabin.pkl', 'wb') as fp:
    pickle.dump(newdict, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [1]:
togeth[togeth.Cabin.isna()]

NameError: name 'togeth' is not defined